In [16]:
from openai import OpenAI
import pandas as pd
from getpass import getpass
from fuzzywuzzy import fuzz
import json

In [17]:
# openai_api_key = getpass("OEPNAI_API_KEY")

# client = OpenAI(api_key=openai_api_key)

In [18]:
voca_index = pd.read_csv('Voca_Index.csv', skiprows=1)
voca_index.copy()
word_list = pd.read_csv('WordList_Mid.csv')
sample_word_list = word_list.loc[word_list['textbook ID'].isin([701])]

In [19]:
sample_word_list.head()

,textbook ID,레슨,단어,품사,PoS,뜻,예문,예문 해석
0,701,7,actually,부사,4,"실제로, 정말로","Actually, exercise can be good for your sleep.",운동은 실제로 숙면에 좋을 수 있다.
1,701,1,add,동사,1,"더하다, 추가하다",Can you add her name to the list?,그녀의 이름을 명단에 추가해 주시겠습니까?
2,701,3,advice,명사,2,"조언, 충고",I need your advice.,당신의 조언이 필요합니다.
3,701,7,amazing,형용사,3,"놀라운, 굉장한",His story was amazing.,그의 이야기는 놀라웠다.
4,701,3,arrive,동사,1,도착하다,We arrived at the airport on time.,우리는 제 시간에 공항에 도착했다.


In [20]:
sample_word_list['품사'].value_counts()

품사
명사            66
동사            46
형용사           30
부사            10
숙어/관용어         3
동사, 명사         2
대명사            1
전치사            1
명사,형용사         1
명사, 형용사        1
동사, 숙어/관용어     1
형용사, 명사        1
동사,숙어/관용어      1
명사, 동사         1
Name: count, dtype: int64

In [21]:
voca_index.columns
drop_cols = ['image ID','WordID(SoundID)', 'WordSymbol']
voca_index.drop(columns=drop_cols, inplace=True)
voca_index = voca_index.rename(columns={
    "WordText": "단어",
    "WordMeaning" : "뜻"
})
voca_index.head()

,voca NO,단어,뜻,PoS,품사
0,1,accent,강세,2,명사
1,2,accent,억양,2,명사
2,3,advertise,광고하다,1,동사
3,4,advertize,광고하다,1,동사
4,5,airline,항공사,2,명사


In [22]:
voca_index['품사'].value_counts()

품사
명사     1318
동사     1060
형용사     372
부사       53
전치사      20
대명사      10
접속사       9
Name: count, dtype: int64

In [23]:
# def AddWord(df_word_list: pd.DataFrame, df_voca_index: pd.DataFrame, output_filename: str) -> str:
#     # voca_options 열을 object 타입으로 설정 (리스트 저장 가능)
#     df_word_list['voca_options'] = pd.Series(dtype='object')

#     for i in range(len(df_word_list)):
#         word = df_word_list.loc[i, '단어']
        
#         # 단어 일치 확인인
#         match_row = df_voca_index[df_voca_index['단어'] == word]

#         # 리스트로 변환하여 저장
#         if not match_row.empty:
#             options = match_row.values.tolist()  # DataFrame -> 리스트 변환

#             # 기존 값 확인 후 리스트 확장
#             existing_value = df_word_list.at[i, 'voca_options']

#             if isinstance(existing_value, list):  # 기존 값이 리스트라면 확장
#                 df_word_list.at[i, 'voca_options'] += options
#             else:  # 기존 값이 없거나 None이면 새 리스트 생성
#                 df_word_list.at[i, 'voca_options'] = options

#     # 🔹 리스트 데이터를 문자열(JSON 형식)로 변환 후 저장
#     df_word_list['voca_options'] = df_word_list['voca_options'].apply(lambda x: json.dumps(x, ensure_ascii=False) if isinstance(x, list) else "")

#     # CSV 파일로 저장
#     df_word_list.to_csv(output_filename, index=False, encoding='utf-8-sig')

#     return f"파일이 저장되었습니다: {output_filename}"


In [24]:
import pandas as pd
import json

def AddWord(df_word_list: pd.DataFrame, df_voca_index: pd.DataFrame, output_filename: str) -> str:
    # voca_options 열을 object 타입으로 설정 (딕셔너리 리스트 저장 가능)
    df_word_list['voca_options'] = pd.Series(dtype='object')

    for i in range(len(df_word_list)):
        word = df_word_list.loc[i, '단어']
        
        # 단어 일치 확인
        match_rows = df_voca_index[df_voca_index['단어'] == word]

        # 일치하는 데이터가 있으면 딕셔너리 리스트로 변환하여 저장
        if not match_rows.empty:
            options = [
                {
                    "voca_id": row[0], 
                    "word": row[1], 
                    "meaning": row[2], 
                    "pos_no": row[3], 
                    "pos": row[4]
                }
                for row in match_rows.itertuples(index=False)
            ]

            # 기존 값 확인 후 리스트 확장
            existing_value = df_word_list.at[i, 'voca_options']

            if isinstance(existing_value, list):  # 기존 값이 리스트라면 확장
                df_word_list.at[i, 'voca_options'] += options
            else:  # 기존 값이 없거나 None이면 새 리스트 생성
                df_word_list.at[i, 'voca_options'] = options

    # 🔹 리스트 데이터를 JSON 문자열로 변환 후 저장
    df_word_list['voca_options'] = df_word_list['voca_options'].apply(
        lambda x: json.dumps(x, ensure_ascii=False) if isinstance(x, list) else ""
    )

    # CSV 파일로 저장
    df_word_list.to_csv(output_filename, index=False, encoding='utf-8-sig')

    return f"파일이 저장되었습니다: {output_filename}"


In [25]:
AddWord(sample_word_list, voca_index, output_filename='sample.csv')

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17264\1662233232.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_list['voca_options'] = pd.Series(dtype='object')
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17264\1662233232.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_list['voca_options'] = df_word_list['voca_options'].apply(


'파일이 저장되었습니다: sample.csv'

In [26]:
# sample_word_list.loc[['voca_options'],:] = None
sample_word_list['voca_options'] = None
sample_word_list.head()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17264\2127395046.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_word_list['voca_options'] = None


,textbook ID,레슨,단어,품사,PoS,뜻,예문,예문 해석,voca_options
0,701,7,actually,부사,4,"실제로, 정말로","Actually, exercise can be good for your sleep.",운동은 실제로 숙면에 좋을 수 있다.,None
1,701,1,add,동사,1,"더하다, 추가하다",Can you add her name to the list?,그녀의 이름을 명단에 추가해 주시겠습니까?,None
2,701,3,advice,명사,2,"조언, 충고",I need your advice.,당신의 조언이 필요합니다.,None
3,701,7,amazing,형용사,3,"놀라운, 굉장한",His story was amazing.,그의 이야기는 놀라웠다.,None
4,701,3,arrive,동사,1,도착하다,We arrived at the airport on time.,우리는 제 시간에 공항에 도착했다.,None


In [27]:
sample_word_list[2:3]

,textbook ID,레슨,단어,품사,PoS,뜻,예문,예문 해석,voca_options
2,701,3,advice,명사,2,"조언, 충고",I need your advice.,당신의 조언이 필요합니다.,None


In [28]:
sample_word_list[160:167]

,textbook ID,레슨,단어,품사,PoS,뜻,예문,예문 해석,voca_options
160,701,1,watch,동사,1,보다,He likes to watch TV all day.,그는 하루 종일 텔레비전 보는 것을 좋아한다.,None
161,701,7,welcome,동사,1,"맞이하다, 환영하다",They came to the station to welcome us.,그들은 우리를 맞이하려고 역으로 나왔다.,None
162,701,5,wet,형용사,3,"젖은, 축축한",I was wet because I fell into the pool.,나는 수영장에 빠져서 젖었다.,None
163,701,5,worry,동사,1,걱정하다[시키다],He worried about his future.,그는 미래에 대해 걱정했다.,None
164,701,4,wrong,형용사,3,"틀린, 잘못된",These phone numbers are wrong in the address b...,주소록에 나와 있는 이 전화번호는 틀린 것이다.,None


In [29]:
sample_word_list['textbook ID'].value_counts()

textbook ID
701    165
Name: count, dtype: int64

In [30]:
for i in range(len(sample_word_list)):
    word = sample_word_list.loc[i, '단어']
    
    # df_voca_index에서 '단어'가 일치하는 모든 행 찾기
    match_row = voca_index[voca_index['단어'] == word]
    print(word, match_row)

actually Empty DataFrame
Columns: [voca NO, 단어, 뜻, PoS, 품사]
Index: []
add Empty DataFrame
Columns: [voca NO, 단어, 뜻, PoS, 품사]
Index: []
advice      voca NO      단어      뜻  PoS  품사
700      701  advice  조언,충고    2  명사
amazing Empty DataFrame
Columns: [voca NO, 단어, 뜻, PoS, 품사]
Index: []
arrive Empty DataFrame
Columns: [voca NO, 단어, 뜻, PoS, 품사]
Index: []
artist Empty DataFrame
Columns: [voca NO, 단어, 뜻, PoS, 품사]
Index: []
behind Empty DataFrame
Columns: [voca NO, 단어, 뜻, PoS, 품사]
Index: []
behind Empty DataFrame
Columns: [voca NO, 단어, 뜻, PoS, 품사]
Index: []
boring Empty DataFrame
Columns: [voca NO, 단어, 뜻, PoS, 품사]
Index: []
borrow Empty DataFrame
Columns: [voca NO, 단어, 뜻, PoS, 품사]
Index: []
both Empty DataFrame
Columns: [voca NO, 단어, 뜻, PoS, 품사]
Index: []
bring Empty DataFrame
Columns: [voca NO, 단어, 뜻, PoS, 품사]
Index: []
broomstick Empty DataFrame
Columns: [voca NO, 단어, 뜻, PoS, 품사]
Index: []
buy Empty DataFrame
Columns: [voca NO, 단어, 뜻, PoS, 품사]
Index: []
camel Empty DataFrame
Columns: [voca 

In [3]:
import pandas as pd
sample = pd.read_csv('sample.csv')
sample['voca_options'].value_counts()

voca_options
[{"voca_id": 1541, "word": "shape", "meaning": "형성하다", "pos_no": 1, "pos": "동사"}, {"voca_id": 1542, "word": "shape", "meaning": "~모양으로 만들다", "pos_no": 1, "pos": "동사"}, {"voca_id": 1543, "word": "shape", "meaning": "상태", "pos_no": 2, "pos": "명사"}, {"voca_id": 1544, "word": "shape", "meaning": "모양,상태", "pos_no": 2, "pos": "명사"}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            2
[{"voca_id": 1644, "word": "strange", "meaning": "낯선", "pos_no": 3, "pos": "형용사"}, {"voca_id": 1645, "word": "strange", "meaning": "이상한", "pos_no": 3, "pos": "형용사"}]